# AutoML Pipeline from ax - Tutorial

We start by importing all the necessary packages.

In [ ]:
import os
import time
from pathlib import Path
from typing import Optional
import json
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pennylane as qml
import torch
from ax import ChoiceParameterConfig, RangeParameterConfig
from matchcake import NonInteractingFermionicDevice
from matchcake.operations import (
    SingleParticleTransitionMatrixOperation,
)
from torchvision.transforms import Resize

from matchcake_opt.datamodules.datamodule import DataModule
from matchcake_opt.modules.classification_model import ClassificationModel
from matchcake_opt.tr_pipeline.automl_pipeline import AutoMLPipeline

Then, we implement our model. In the next cell, you will see an example of classification model from the lightning tutorial. The only difference now is that we specified the hyperparameter configuration of our model in the static attribute `HP_CONFIGS`. It's really important that you override this attribute for all your model since the `AutoMLPipeline` will use it to learn the best hyperparameters set for your model.

In [ ]:
class NIFCNN(ClassificationModel):
    """
    Non-Interacting fermions classifier through CNN hamiltonian encoding. The flow of information
    in the model goes as follows.

    1. CNN(X_{bchw}) -> X_{be}: Build the embeddings with the CNN encoder
    2. Linear(X_{be}) -> W_{bn}: Build the local fields weights from the embeddings
    3. Linear(X_{be}) -> W_{bt}: Build the ZZ couplings weights from the embeddings
    - We now have the hamiltonian: H(X_{be}) = W_{bn} Z_{n} + W_{bt} ZZ_{t}
    4. P_{knu}, P_{ktv}: We compute the probabilities to get the eigenstates of Z and ZZ
    5. E_{k} = W_{bn} P_{knu} Lambda_{u} + W_{bt} P_{ktv} Lambda_{v}: We now compute the expected values with Lambda_{u} and Lambda_{v} the eigenvalues of Z and ZZ respectively.
    6. Softmax(E_{k}) -> Y_{k}: Finally, we apply a softmax to get the probabilities of each class.
    """
    MODEL_NAME = "NIFCNN"
    DEFAULT_N_QUBITS = 16
    DEFAULT_LEARNING_RATE = 2e-4
    DEFAULT_ENCODER_OUTPUT_ACTIVATION = "Tanh"
    MIN_INPUT_SIZE = (28, 28)

    HP_CONFIGS = [
        RangeParameterConfig(
            name="learning_rate",
            parameter_type="float",
            bounds=(1e-5, 0.1),
        ),
        RangeParameterConfig(
            name="n_qubits",
            parameter_type="int",
            bounds=(4, 92),
        ),
        ChoiceParameterConfig(
            name="encoder_output_activation",
            parameter_type="str",
            values=["Tanh", "Identity"],
            is_ordered=False,
        ),
    ]

    def __init__(
            self,
            input_shape: Optional[tuple[int, ...]],
            output_shape: Optional[tuple[int, ...]],
            learning_rate: float = DEFAULT_LEARNING_RATE,
            n_qubits: int = DEFAULT_N_QUBITS,
            encoder_output_activation: str = DEFAULT_ENCODER_OUTPUT_ACTIVATION,
            **kwargs,
    ):
        super().__init__(input_shape=input_shape, output_shape=output_shape, learning_rate=learning_rate, **kwargs)
        self.save_hyperparameters("learning_rate", "n_qubits", "encoder_output_activation")
        self.n_qubits = n_qubits
        self.R_DTYPE = torch.float32
        self.C_DTYPE = torch.cfloat
        self._n_params = np.triu_indices(2 * self.n_qubits, k=1)[0].size
        self.q_device = NonInteractingFermionicDevice(
            wires=self.n_qubits, r_dtype=self.R_DTYPE, c_dtype=self.C_DTYPE, show_progress=False
        )
        self.encoder_output_activation = encoder_output_activation
        self.input_resize = Resize(self.MIN_INPUT_SIZE)
        self.local_fields_encoder = torch.nn.Sequential(
            torch.nn.LazyConv2d(512, kernel_size=7),
            torch.nn.LazyBatchNorm2d(),
            torch.nn.LazyConv2d(128, kernel_size=5),
            torch.nn.LazyBatchNorm2d(),
            torch.nn.LazyConv2d(64, kernel_size=3),
            torch.nn.LazyBatchNorm2d(),
        )

        self.local_fields_head = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.LazyLinear(self.n_qubits),
            getattr(torch.nn, encoder_output_activation)()
        )
        self._hamiltonian_n_params = np.triu_indices(self.n_qubits, k=1)[0].size
        self.zz_body_couplings_head = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.LazyLinear(self._hamiltonian_n_params),
            getattr(torch.nn, encoder_output_activation)()
        )

        self.zz_body_coupling_weights = torch.nn.Parameter(torch.randn(self._hamiltonian_n_params), requires_grad=True)

        self.local_fields_op_eigvals = torch.nn.Parameter(torch.from_numpy(np.array([1.0, -1.0])).float(),
                                                          requires_grad=False)  # eigvals(Z)
        self.zz_eigvals = torch.nn.Parameter(torch.from_numpy(np.array([1.0, -1.0, -1.0, 1.0])).float(),
                                             requires_grad=False)  # eigvals(ZZ)

        self.local_fields_wires = [[i] for i in range(self.n_qubits)]
        self.couplings_wires = [[i, j] for i, j in np.vstack(np.triu_indices(self.n_qubits, k=1)).T]

        self.weights = torch.nn.Parameter(torch.rand((int(self.output_size), self._n_params)), requires_grad=True)
        torch.nn.init.xavier_uniform_(self.weights)
        self._build()

    def _build(self):
        dummy_input = torch.randn((3, *self.input_shape)).to(device=self.device)
        with torch.no_grad():
            self(dummy_input)
        return self

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.preprocess_input(x)
        embeddings = self.local_fields_encoder(x)

        local_fields = self.local_fields_head(embeddings)
        zz_couplings = self.zz_body_couplings_head(embeddings)
        self.q_device.execute_generator(self.circuit_gen(), reset=True)

        local_fields_probs = (
            self.q_device.probability(wires=self.local_fields_wires)
            .to(dtype=self.q_device.R_DTYPE, device=self.torch_device)
        )
        couplings_probs = (
            self.q_device.probability(wires=self.couplings_wires)
            .to(dtype=self.q_device.R_DTYPE, device=self.torch_device)
        )
        weighted_local_eigvals = torch.einsum(
            "bi,kij,j->bk", local_fields, local_fields_probs, self.local_fields_op_eigvals
        )
        weighted_zz_eigvals = torch.einsum(
            "bi,kij,j->bk", zz_couplings, couplings_probs, self.zz_eigvals
        )

        expval = weighted_local_eigvals + weighted_zz_eigvals
        return expval

    def circuit_gen(self):
        yield qml.BasisState(self.initial_basis_state, wires=self.wires)
        yield self.get_sptm_weights()
        return

    def get_sptm_weights(self):
        """
        Compute the single-particle transition matrix (SPTM) weights based on the initialized weight tensor.

        This method constructs a tensor, `h`, which encodes the pairwise weight interactions
        in a prescribed upper triangular form. It then symmetrizes `h` to ensure anti-symmetry
        about the main diagonal. Afterward, the matrix exponential of `h` is computed to generate
        the SPTM. Finally, the SPTM is encapsulated in a `SingleParticleTransitionMatrixOperation`
        object for further usage.

        :return: An instance of `SingleParticleTransitionMatrixOperation` that encapsulates
            the computed single-particle transition matrix based on the initialized weights.
            The shape of the SPTM is (n_classes, 2 * n_qubits, 2 * n_qubits).
        :rtype: SingleParticleTransitionMatrixOperation
        """
        h = torch.zeros(
            (int(self.output_size), 2 * self.n_qubits, 2 * self.n_qubits), dtype=self.R_DTYPE, device=self.torch_device
        )
        triu_indices = np.triu_indices(2 * self.n_qubits, k=1)
        h[:, triu_indices[0], triu_indices[1]] = self.weights
        h = h - h.mT
        sptm = torch.matrix_exp(h)
        return SingleParticleTransitionMatrixOperation(sptm, wires=self.wires)

    def preprocess_input(self, x: torch.Tensor, **kwargs) -> torch.Tensor:
        x = x.reshape(x.shape[0], -1, x.shape[-2], x.shape[-1])
        if x.shape[-2] < self.MIN_INPUT_SIZE[-2] or x.shape[-1] < self.MIN_INPUT_SIZE[-1]:
            x = self.input_resize(x)
        return x

    @property
    def torch_device(self):
        return self.device

    @property
    def wires(self):
        return self.q_device.wires

    @property
    def initial_basis_state(self):
        return np.zeros(self.n_qubits, dtype=int)

    @property
    def output_size(self):
        return int(np.prod(self.output_shape))


Now we define our hyperparameters. Notice this time, we don't have any hyperparameters for our model. This is normal since our goal is to optimize those through the automl pipeline.

In [ ]:
# Dataset
dataset_name = "Digits2D"
fold_id = 0
batch_size = 32
random_state = 0
num_workers = 0

# Model
model_cls = NIFCNN

# Pipeline
job_output_folder = Path(os.getcwd()) / "data" / "automl" / dataset_name / model_cls.MODEL_NAME
checkpoint_folder = Path(job_output_folder) / "checkpoints"
pipeline_args = dict(
    max_epochs=100,  # increase at least to 256
    max_time="00:00:02:00",  # DD:HH:MM:SS, increase at least to "00:01:00:00"
)

Now, we instantiate our datamodule and our automl pipline. Here some explanation on some of the parameters:

- `automl_iterations`: That's the number of hyperparameters configuration the pipeline will try before choosing the best one.
- `inner_max_epochs`: That's the number of epochs each configuration will be trained for.
- `inner_max_time`: That's the maximum time each configuration will be trained for.

In [ ]:
datamodule = DataModule.from_dataset_name(
    dataset_name,
    fold_id=fold_id,
    batch_size=batch_size,
    random_state=random_state,
    num_workers=num_workers,
)
automl_pipeline = AutoMLPipeline(
    model_cls=model_cls,
    datamodule=datamodule,
    checkpoint_folder=checkpoint_folder,
    automl_iterations=5,  # increase at least to 32
    inner_max_epochs=10,  # increase at least to 128
    inner_max_time="00:00:01:00",  # increase at least to "00:00:10:00"
    automl_overwrite_fit=True,
    **pipeline_args
)

It's time to run the optimization.

In [ ]:
start_time = time.perf_counter()
automl_pipeline.run()
end_time = time.perf_counter()
print(f"Time taken: {end_time - start_time:.4f} seconds")

We can then check the best hyperparameters found.

In [ ]:
print(f"Best Hyperparameters:\n{json.dumps(automl_pipeline.get_best_params(), indent=2, default=str)}")

Finally, we can run a full training on the best configuration found.

In [ ]:
lt_pipeline, metrics = automl_pipeline.run_best_pipeline()
print("⚡" * 20, "\nValidation Metrics:\n", metrics, "\n", "⚡" * 20)

And run the test evaluation.

In [ ]:
test_metrics = lt_pipeline.run_test()
print("⚡" * 20, "\nTest Metrics:\n", test_metrics, "\n", "⚡" * 20)

----------------------------------------